In [1]:
#GENERATING AUDIENCE DATA
import time
import requests
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta

#datas a serem requeridas:
start_date='2021-01' #YYYY-MM
end_date='2023-09'  #YYYY-MM

start = datetime.strptime(start_date, "%Y-%m")
end = datetime.strptime(end_date, "%Y-%m")

def process_dataframe(input_df, company, vertical, state,platform):
    # Create a new DataFrame with only the desired columns
    processed_df = input_df[['date', 'visits', 'bounce_rate', 'pages_per_visit',
                             'visit_duration', 'page_views', 'share', 'unique_visitors']].copy()
    
    # Add new columns with the provided inputs
    processed_df['company'] = company
    processed_df['vertical'] = vertical
    processed_df['state'] = state
    processed_df['platform'] = platform
    
    return processed_df

def transform_dataframe(original_df, company, vertical, state, platform):
    metric_name = original_df.columns[1]
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[metric_name]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

def transform_dataframe_0(original_df, company, vertical, state, platform,metric_name):
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[original_df.columns[0]]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

def add_data_to_dataframe(df, json_data,age,gender,company,vertical):
    # Extrai os valores do JSON
    start_date = json_data['meta']['request']['start_date']
    share = json_data['share']
    pages_per_visit = json_data['pages_per_visit']
    bounce_rate = json_data['bounce_rate']
    visit_duration = json_data['visit_duration']

    # Adiciona os valores ao DataFrame
    df = df.append({
        'date': start_date,
        'audience_share': share,
        'pages_per_visit': pages_per_visit,
        'bounce_rate': bounce_rate,
        'visit_duration': visit_duration,
        'age': age,
        'gender': gender,
        'company': company,
        'vertical': vertical
    }, ignore_index=True)
    return df

company_urls =[
'olx.com.br', 
'quintoandar.com.br',
'chavesnamao.com.br',
'icarros.com.br',
'imovelweb.com.br',
'kavak.com',
'loft.com.br',
'mobiauto.com.br',
'vivareal.com.br',
'webmotors.com.br',
'zapimoveis.com.br',
'casamineira.com.br', 
'meucarronovo.com.br',
'enjoei.com.br',
'elo7.com.br',
'mercadolivre.com.br',
'shopee.com.br'
]
company_names =[
    'olx',
'quinto_andar',
'chaves_na_mao',
'icarros',
'imovelweb',
'kavak',
'loft',
'mobiauto',
'vivareal',
'webmotors',
'zap',
'casa_mineira',
'meu_carro_novo',
'enjoei',
'elo7',
'mercado_livre',
'shopee'
]
vertical_names =[
'FULL',
'REAL_ESTATE',
'FULL',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'REAL_ESTATE',
'AUTOS',
'GOODS',
'GOODS',
'GOODS',
'GOODS'
]

device_names =[
'Total',
'Desktop',
'Mobile-web'
]
api_device_names =[
'total-audience-interests',
'audience-interests',
'mobile-audience-interests'
]
company_map = dict(zip(company_urls, company_names)) #mapear uma lista na outra
vertical_map = dict(zip(company_urls, vertical_names)) #mapear uma lista na outra
device_map = dict(zip(device_names, api_device_names)) #mapear uma lista na outra

# Replace with your actual API key
API_KEY = "c165c953a9fd44908addfcccec2c8caa"
# Headers with authorization
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

#dataframe das métricas
columns = ['dt','company_from','company_to', 'cross_visitors','vertical','device','month','year']
metrics_df =pd.DataFrame(columns=columns)
granularity='monthly'
sleep_time=0.4
#################################################################################################################
while start <= end:
    first_day = start.strftime("%Y-%m-01")
    last_day = (start + relativedelta(day=31)).strftime("%Y-%m-%d")
                                                    ####################
                                                    ###Loop de coleta###
    for company_url in company_urls:                ####################
        company_from=company_map.get(company_url)
        vertical=vertical_map.get(company_url)
        for device in device_names:
           
            #total
            url='https://api.similarweb.com/v4/website/'+company_url+'/'+device_map.get(device)+'/also-visited?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+first_day+'&end_date='+last_day+'&country=br&main_domain_only=false&format=json' 
            # Make the API request
            response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
            # Check if the request was successful
            if response.status_code == 200:
                data= response.json()
                # Process the data as needed
                print('Data for '+company_from+'/'+device+'/'+str(start)+' overlap processed successfully!')
            else:
                print('API request failed for '+company_from+' :', response.status_code)
                
            # Extract 'overlap' and 'domain' pairs
            overlap_list = [record['overlap'] for record in data['records']]
            domain_list = [record['domain'] for record in data['records']]
            date = data['meta']['request']['start_date']

            # Create a DataFrame with three columns
            df = pd.DataFrame({'cross_visitors': overlap_list, 'company_to': domain_list, 'dt': [date] * len(overlap_list)})
            df['dt'] = pd.to_datetime(df['dt'])
            df['year'] = df['dt'].dt.year.astype('int32')
            df['month'] = df['dt'].dt.month.astype('int32')
            df['company_from'] = company_from
            df['vertical'] = vertical
            df['device'] = device
            metrics_df=pd.concat([metrics_df, df], ignore_index=True) # apenda novos dados
            time.sleep(sleep_time)
    start = start + relativedelta(months=1)
df=metrics_df
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.astype({col:'string'})
for col in df.columns:
    if df[col].dtype == 'float64':
        df = df.astype({col:'float32'})

metrics_df=df
metrics_df['month'] = metrics_df['month'].astype('int32')
metrics_df['year'] = metrics_df['year'].astype('int32')

# Exibe o DataFrame resultante
print(metrics_df)

Data for olx/Total/2021-01-01 00:00:00 overlap processed successfully!
Data for olx/Desktop/2021-01-01 00:00:00 overlap processed successfully!
Data for olx/Mobile-web/2021-01-01 00:00:00 overlap processed successfully!
Data for quinto_andar/Total/2021-01-01 00:00:00 overlap processed successfully!
Data for quinto_andar/Desktop/2021-01-01 00:00:00 overlap processed successfully!
Data for quinto_andar/Mobile-web/2021-01-01 00:00:00 overlap processed successfully!
Data for chaves_na_mao/Total/2021-01-01 00:00:00 overlap processed successfully!
Data for chaves_na_mao/Desktop/2021-01-01 00:00:00 overlap processed successfully!
Data for chaves_na_mao/Mobile-web/2021-01-01 00:00:00 overlap processed successfully!
Data for icarros/Total/2021-01-01 00:00:00 overlap processed successfully!
Data for icarros/Desktop/2021-01-01 00:00:00 overlap processed successfully!
Data for icarros/Mobile-web/2021-01-01 00:00:00 overlap processed successfully!
Data for imovelweb/Total/2021-01-01 00:00:00 overla

In [33]:
metrics_df.to_csv('similarweb_audience-'+start_date+'-'+end_date+'.csv', index=False)

In [18]:
import io
import os

#import boto3
import s3fs

import pandas as pd
import pyarrow as pa

In [19]:
from dataeng_services.session import Session

session_provider = Session().client("session-provider")
aws_auth_dict = session_provider.get_temporary_credentials().json()

In [20]:
AWS_S3_FOLDER = 's3://olxbr-dl-cross-estrategia-e-planejamento/prod/cross_strategy/similarweb_audience'
# csv_file_name = 'dataframe_s'

date_cols = ['dt']
int_cols = []
partition_col = ['dt']

In [21]:
#LENDO ARQUIVO ORIGINAL
df = metrics_df
# df = pd.read_csv(csv_file_name+'.csv')

df.head()

,dt,company_from,company_to,cross_visitors,vertical,device,month,year
0,2021-01-01,olx,produto.mercadolivre.com.br,0.287838,FULL,Total,1,2021
1,2021-01-01,olx,mercadolivre.com.br,0.242313,FULL,Total,1,2021
2,2021-01-01,olx,lista.mercadolivre.com.br,0.288112,FULL,Total,1,2021
3,2021-01-01,olx,facebook.com,0.410625,FULL,Total,1,2021
4,2021-01-01,olx,youtube.com,0.666488,FULL,Total,1,2021


In [22]:
# Coverte as colunas de data
for col in date_cols:
    df[ col ] = pd.to_datetime(df[ col ], format='%Y-%m-%d') ## Date   

# Coverte as colunas int
df[ int_cols ] = df[ int_cols ].round()

# Confirma os tipos corretos das Var na AWS
pa_schema = pa.Schema.from_pandas(df)
pa_schema = pa.schema([
pa.field(field.name, pa.date32()) if field.name in date_cols else field
for field in pa_schema
])

In [23]:
#Convertendo e salvando na aws
# save converted files localy
if len( partition_col ) > 0 :
    df.to_parquet(
            AWS_S3_FOLDER,
            schema=pa_schema,
            partition_cols=partition_col,
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )
else:
    df.to_parquet(
            AWS_S3_FOLDER+csv_file_name+'.parquet', 
            schema=pa_schema, 
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )

In [24]:
#adicionando partições à metastore
from dataeng_services.session import Session
from datetime import datetime,timedelta

# In order to avoid exposing credentials in your code
# You may use dataeng-cli --refresh-token first to create/refresh a token
# Or provide env variables OLXBR_INTERNAL_AUTH_USERNAME and OLXBR_INTERNAL_AUTH_PASSWORD

from dataeng_services.session import Session

metaservice = Session().client("metaservice")

In [25]:
# Dados do processo no ETLaaS
from dateutil.relativedelta import *
schema_name = 'cross_strategy'
table_name = 'similarweb_audience'

# Periodo
start_date = '2021-01-01'
end_date = '2023-12-01'
date_step = relativedelta(months=+1) # mensal
# date_step = timedelta(days = 1)  # diario

In [26]:
date_list = []
day = start_date
while datetime.strptime( day, '%Y-%m-%d') <= datetime.strptime( end_date, '%Y-%m-%d'):
    date_list.append( day )
    day = str( datetime.strptime( day, '%Y-%m-%d') + date_step )[:10]
#     day = day[:8]+'01' # mensal
# dt=2023-02-01/
    
date_list

['2021-01-01',
 '2021-02-01',
 '2021-03-01',
 '2021-04-01',
 '2021-05-01',
 '2021-06-01',
 '2021-07-01',
 '2021-08-01',
 '2021-09-01',
 '2021-10-01',
 '2021-11-01',
 '2021-12-01',
 '2022-01-01',
 '2022-02-01',
 '2022-03-01',
 '2022-04-01',
 '2022-05-01',
 '2022-06-01',
 '2022-07-01',
 '2022-08-01',
 '2022-09-01',
 '2022-10-01',
 '2022-11-01',
 '2022-12-01',
 '2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01']

In [27]:
# Adding partitions
response = metaservice.add_partition(
    schema_name=schema_name,
    table_name=table_name,
    partition_names=partition_col,
    partition_values=[ [i] for i in date_list],
)

# Response
response.content

b'{"detail":"The partitions have been added to the requested table!"}'

In [28]:
print(metrics_df)

               dt company_from                   company_to  cross_visitors  \
0      2021-01-01          olx  produto.mercadolivre.com.br        0.287838   
1      2021-01-01          olx          mercadolivre.com.br        0.242313   
2      2021-01-01          olx    lista.mercadolivre.com.br        0.288112   
3      2021-01-01          olx                 facebook.com        0.410625   
4      2021-01-01          olx                  youtube.com        0.666488   
...           ...          ...                          ...             ...   
149932 2023-09-01       shopee             boticario.com.br        0.005733   
149933 2023-09-01       shopee                    remove.bg        0.006133   
149934 2023-09-01       shopee        lojadomecanico.com.br        0.004582   
149935 2023-09-01       shopee                serasa.com.br        0.011965   
149936 2023-09-01       shopee                       gov.br        0.032184   

       vertical      device  month  year  
0       

In [29]:
df=metrics_df
for column in df.columns:
    print(f"Column Name: {column}, Data Type: {df[column].dtype}")

Column Name: dt, Data Type: datetime64[ns]
Column Name: company_from, Data Type: string
Column Name: company_to, Data Type: string
Column Name: cross_visitors, Data Type: float32
Column Name: vertical, Data Type: string
Column Name: device, Data Type: string
Column Name: month, Data Type: int32
Column Name: year, Data Type: int32


In [30]:
metrics_df['year'] = metrics_df['year'].astype('int32')

In [31]:
metrics_df['month'] = metrics_df['month'].astype('int32')
metrics_df['year'] = metrics_df['year'].astype('int32')